In [1]:
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
from keras import layers
import keras
import numpy as np

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt

from numpy.random import seed
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

def create_sequences(values, time_steps=128):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

data = pd.read_csv(f'data/SKAB/anomaly-free/anomaly-free.csv', sep = ';')
scaler = StandardScaler()


data.index = pd.to_datetime(data['datetime'])
data.drop('datetime', axis = 1, inplace=True)

#train = data[data['anomaly'] == 0]
train = data.copy()
train = train.values
scaler.fit(train)
train = scaler.transform(train)

pca = PCA(n_components=2)
train = pca.fit_transform(train)

#test = data[data['anomaly'] == 1]

#test.drop(['anomaly', 'changepoint'], axis = 1, inplace = True)
#test = test.values
#test = scaler.transform(test)
#time_steps = dataset_size - 1

X = create_sequences(train, time_steps=59)
#X_test = create_sequences(test, time_steps=time_steps)


model = keras.Sequential(
        [
            layers.Input(shape=(X.shape[1], X.shape[2])),#, X.shape[3])),

            #layers.Dropout(rate=0.2),
            layers.Conv1D(
                filters=32, kernel_size=7, padding="same", strides=1, activation="relu"
            ),

            # layers.Dropout(rate=0.2),
            layers.Conv1D(
                filters=16, kernel_size=7, padding="same", strides=1, activation="relu"
            ),

            layers.Conv1DTranspose(
                filters=16, kernel_size=7, padding="same", strides=1, activation="relu"
            ),

            layers.Conv1DTranspose(
                filters=32, kernel_size=7, padding="same", strides=1, activation="relu"
            ),
            layers.Conv1DTranspose(
                filters=1, kernel_size=7, padding="same"),

            #layers.Dense(X.shape[2])
        ]
        )


model.compile(optimizer='adam', loss='mse')

n_epochs = 100

hist =  model.fit(X, X,
        epochs = n_epochs, batch_size = 8)

model.save_weights('./checkpoints/my_checkpoint_PCA')


2023-10-28 16:03:48.583472: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-28 16:03:48.583491: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/100


2023-10-28 16:03:51.929155: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-28 16:03:51.929462: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-28 16:03:51.929524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-10-28 16:03:51.929575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-10-28 16:03:51.929625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

1169/1169 [==============================] - 3s 2ms/step - loss: 1.3526
Epoch 2/100
1169/1169 [==============================] - 2s 2ms/step - loss: 1.3331
Epoch 3/100
1169/1169 [==============================] - 2s 2ms/step - loss: 1.3349
Epoch 4/100
1169/1169 [==============================] - 3s 2ms/step - loss: 1.3309
Epoch 5/100
1169/1169 [==============================] - 3s 2ms/step - loss: 1.3320
Epoch 6/100
1169/1169 [==============================] - 3s 2ms/step - loss: 1.3310
Epoch 7/100
1169/1169 [==============================] - 2s 2ms/step - loss: 1.3314
Epoch 8/100
1169/1169 [==============================] - 2s 2ms/step - loss: 1.3306
Epoch 9/100
1169/1169 [==============================] - 2s 2ms/step - loss: 1.3320
Epoch 10/100
1169/1169 [==============================] - 2s 2ms/step - loss: 1.3311
Epoch 11/100
1169/1169 [==============================] - 2s 2ms/step - loss: 1.3308
Epoch 12/100
1169/1169 [==============================] - 3s 2ms/step - loss: 1.3308
E

In [2]:
hist.history['loss']

[1.3525806665420532,
 1.3331427574157715,
 1.334916591644287,
 1.330893635749817,
 1.3320480585098267,
 1.3309773206710815,
 1.3314381837844849,
 1.330625295639038,
 1.3320090770721436,
 1.3311289548873901,
 1.3308128118515015,
 1.330759048461914,
 1.3308826684951782,
 1.3305964469909668,
 1.3313287496566772,
 1.3305145502090454,
 1.3308238983154297,
 1.33154296875,
 1.3303955793380737,
 1.3307350873947144,
 1.330772042274475,
 1.3315699100494385,
 1.3303616046905518,
 1.3305137157440186,
 1.3309760093688965,
 1.330513834953308,
 1.3304036855697632,
 1.3308550119400024,
 1.3306711912155151,
 1.3306193351745605,
 1.3306231498718262,
 1.3309824466705322,
 1.330346941947937,
 1.3304684162139893,
 1.33060884475708,
 1.3306779861450195,
 1.330481767654419,
 1.3304311037063599,
 1.3304426670074463,
 1.3305022716522217,
 1.330541968345642,
 1.330621361732483,
 1.3305466175079346,
 1.3308091163635254,
 1.3305143117904663,
 1.3303580284118652,
 1.3304989337921143,
 1.3308099508285522,
 1.330322